In [3]:
from fastai.learner import load_learner
import cv2
import os
import random
from PIL import Image
import numpy as np
from matplotlib import image
import matplotlib.pyplot as plt

In [2]:
def mask_from_image(fname): 
    fname = str(fname).replace('images','labels').replace('.jpg', "_train_id.png")
    
    return fname

def acc(input, target):
    target = target.squeeze(1)
    mask = target != 20
    return (input.argmax(dim=1)[mask] == target[mask]).float().mean()


def highest_pixels(img, threshold=0):
    highest = []
    for i in range(img.shape[1]):
        column = img[:, i]
        wanted = np.where(column > threshold)[0]
        if len(wanted) == 0:
            highest.append(0)
        else:
            highest.append(wanted[0])
            
    return highest

def crop_black_top(img, highest_road_pixels, height):
    cutoff_at = min(np.min(highest_road_pixels), int(height / 2.5))
    height_multiplyer = img.shape[0] / height
    cutoff_at = int(cutoff_at * height_multiplyer)

    return img[cutoff_at:, :, :]

def crop_top(img):
    highest_road_pixels = [p for p in highest_pixels(img) if p > 0]

    if len(highest_road_pixels) == 0:
        return img

    return crop_black_top(img, highest_road_pixels, img.shape[0])


def expand_mask(binary_mask):
    mask = cv2.dilate(np.invert(cv2.medianBlur(binary_mask, 103)), cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (30,30)))
    
    
    if np.sum(mask) < np.prod(mask.shape) / 3 * 255:
        mask = cv2.dilate(mask, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (140,140)))
        
    mask = (mask / 255).astype("uint8")
    
    return mask
    
def hide_non_road(img, mask):
    return np.expand_dims(cv2.resize(mask, dsize=img.shape[:2]), 2) * img

def road_preds(preds):
    classes = preds[0]
    
    classes = np.where(classes==20, 0, classes)
    classes = np.where(classes==1, 0, classes)
    classes = np.where(classes==13, 0, classes)
    classes = np.where(classes==14, 0, classes)
    classes = np.where(classes==15, 0, classes)
    classes = np.where(classes==17, 0, classes)
    
    return (np.where(classes>0, 1, classes) * 255).astype("uint8")

class UnetModelWrapper():
    def __init__(self, model_path) -> None:
        self.loaded = False 
        self.model_path = model_path
        self.model = None
    
    def __call__(self, img):
        if not self.loaded:
            self.model = load_learner(self.model_path, cpu=False)
            self.loaded = True
        
        original_size = (img.shape[1], img.shape[0])

        mask = expand_mask(road_preds(self.model.predict(img)))
        masked_img = hide_non_road(img, mask)
        return masked_img

In [3]:
def full_contents(path):
    return [path + f for f in os.listdir(path)]

In [4]:
def get_imgs_r(directory, suffixes=[".jpg", ".jpeg"]):
    imgs = []
    
    if not os.path.isdir(directory):
        if any([s in directory for s in suffixes]):
            return [directory]
        return []
    
    directory += "/"
    
    children = full_contents(directory)
    
    for child in children:
        imgs.extend(get_imgs_r(child))
    
    return imgs
    

In [21]:
all_imgs = []

img_folders = [
    "/home/lewington/code/rddc2020/yolov5/datasets/road2020/train",
    "/home/lewington/code/rddc2020/yolov5/yolov5/road2020/datasets/test1",
    "/home/lewington/code/rddc2020/yolov5/yolov5/road2020/datasets/test2"
]

for folder in img_folders:
    all_imgs.extend(get_imgs_r(folder))

In [ ]:
unet = UnetModelWrapper("/home/lewington/code/faultnet/app/unet/models/frozen-15epochs")

for path in all_imgs:
    data = image.imread(path)
    cropped = unet(data)    
    
    Image.fromarray(cropped).save(path)
    print(path)

/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_006048.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_012129.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_004371.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_007484.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_003016.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_004246.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_007740.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_002852.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_011857.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_001043.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_009614.jpg

/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_002586.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_000852.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_004706.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_000035.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_006501.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_010868.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_005696.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_009707.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_004379.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_011512.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_003853.jpg

/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_007723.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_008539.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_009392.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_006090.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_007410.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_008074.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_009668.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_002357.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_004494.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_010890.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_008369.jpg

/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_005993.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_011183.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_012580.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_009973.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_011747.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_001259.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_011479.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_003035.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_010992.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_010436.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_008712.jpg

/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_004237.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_012646.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_009539.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_007851.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_005668.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_010038.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_012673.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_008493.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_007015.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_007074.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_000576.jpg

...

/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_011339.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_011198.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_005529.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_008667.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_007059.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_004759.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_005970.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_005320.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_001944.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_002616.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_007396.jpg

/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_004513.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_001472.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_003759.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_006555.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_011854.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_009784.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_008196.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_011699.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_002691.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_003240.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_000066.jpg

/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_005658.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_012683.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_001243.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_002823.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_009022.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_004525.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_003209.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_004619.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_012498.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_008321.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_012390.jpg

/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_004610.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_000933.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_004941.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_011078.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_009779.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_010479.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_008494.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_010208.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_009525.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_009556.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_002440.jpg

/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_012196.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_012330.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_002799.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_010838.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_001519.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_012961.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_005262.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_002122.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_009212.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_012944.jpg
/home/lewington/code/rddc2020/yolov5/datasets/road2020/train/Japan/images/Japan_007081.jpg

In [8]:
Image.fromarray(y).save("lel.jpg")

In [17]:
q = all_imgs("yolov5")

In [ ]:
def remake_dataset(output_dir, data_dirs):
    